# ✔ CUDA setup

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
!nvidia-smi

Thu Feb  3 20:22:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hidoc2rt
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hidoc2rt
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=5a4832b9f99ec63ec8b4c7bee778896ad805cc256740fcc74793c07141523054
  Stored in directory: /tmp/pip-ephem-wheel-cache-hvz4faao/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# ✔ VS Code on Colab

In [ ]:
# 1. Install the colab-code package...
!pip install colabcode

In [ ]:
# 2. Import and launch...
from colabcode import ColabCode
ColabCode()

Code Server can be accessed on: NgrokTunnel: "https://1636-35-185-135-22.ngrok.io" -> "http://localhost:10000"
[2021-10-16T17:00:56.429Z] info  code-server 3.10.2 387b12ef4ca404ffd39d84834e1f0776e9e3c005
[2021-10-16T17:00:56.431Z] info  Using user-data-dir ~/.local/share/code-server
[2021-10-16T17:00:56.450Z] info  Using config file ~/.config/code-server/config.yaml
[2021-10-16T17:00:56.450Z] info  HTTP server listening on http://127.0.0.1:10000 
[2021-10-16T17:00:56.450Z] info    - Authentication is disabled 
[2021-10-16T17:00:56.450Z] info    - Not serving HTTPS 
 INFO Installing extension: ms-vscode.cpptools
 INFO Downloaded extension: ms-vscode.cpptools /root/.local/share/code-server/CachedExtensionVSIXs/ms-vscode.cpptools-1.5.1
 INFO Extracted extension to /root/.local/share/code-server/extensions/.801240b6-e04b-4087-9084-3b6369662ed5: ms-vscode.cpptools
 INFO Renamed to /root/.local/share/code-server/extensions/ms-vscode.cpptools-1.5.1
 INFO Installation completed. ms-vscode.cppt

#DeviceQuery

In [ ]:
#@title working directory: **deviceQuery/**
%cd /home/grossi/CUDA/lab4/deviceQuery/
%ls

In [ ]:
%%writefile /home/grossi/CUDA/lab4/deviceQuery/helper.h
// Beginning of GPU Architecture definitions
inline int _ConvertSMVer2Cores(int major, int minor) {
	// Defines for GPU Architecture types (using the SM version to determine
	// the # of cores per SM
	typedef struct {
		int SM;  // 0xMm (hexidecimal notation), M = SM Major version,
		// and m = SM minor version
		int Cores;
	} sSMtoCores;

	sSMtoCores nGpuArchCoresPerSM[] = {
			{0x20, 32},
			{0x30, 192},
			{0x32, 192},
			{0x35, 192},
			{0x37, 192},
			{0x50, 128},
			{0x52, 128},
			{0x53, 128},
			{0x60,  64},
			{0x61, 128},
			{0x62, 128},
			{0x70,  64},
			{0x72,  64},
			{0x75,  64},
			{-1, -1}};

	int index = 0;

	while (nGpuArchCoresPerSM[index].SM != -1) {
		if (nGpuArchCoresPerSM[index].SM == ((major << 4) + minor)) {
			return nGpuArchCoresPerSM[index].Cores;
		}

		index++;
	}

	// If we don't find the values, we default use the previous one
	// to run properly
	printf(
			"MapSMtoCores for SM %d.%d is undefined."
			"  Default to use %d Cores/SM\n",
			major, minor, nGpuArchCoresPerSM[index - 1].Cores);
	return nGpuArchCoresPerSM[index - 1].Cores;
}


In [ ]:
%%writefile /home/grossi/CUDA/lab4/deviceQuery/deviceQuery.cu
#include <stdlib.h>
#include <stdio.h>
#include "helper.h"
#include "../../utils/common.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

	    printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
	           deviceProp.multiProcessorCount,
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
	               deviceProp.multiProcessorCount);

		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory:               %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block:       %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block: %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size:                                     %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor:  %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block:           %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch:                          %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment:                             %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution:          %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels:                     %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory:            %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping:       %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces:            %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support:                        %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc deviceQuery.cu -o deviceQuery
!deviceQuery

In [9]:
# Occupancy

!nvprof --metrics achieved_occupancy ./FlockingBehaviourPar

======== Error: Application returned non-zero code -1


# Flocking Behaviour Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential"
%ls

/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential
Boid.cpp  Flock.cpp  FlockingBehaviourSeq     FlockingBehaviourSeq.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourSeq.cu  src/                    Utils.h


In [41]:
%cd ..
%cd "FlockingBehaviourParallel"
%ls

/content/drive/MyDrive/GPU Computing/Progetto
/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel
Boid.cpp   Flock.h                  FlockingBehaviourPar.h   Utils.cpp
Boid.h     FlockingBehaviourPar*    FlockingBehaviourSeq.cu  Utils.h
Flock.cpp  FlockingBehaviourPar.cu  src/


In [37]:
%cd ..
%cd "FlockingBehaviourSequential"
%ls

/content/drive/MyDrive/GPU Computing/Progetto
/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential
Boid.cpp  Flock.cpp  FlockingBehaviourSeq*    FlockingBehaviourSeq.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourSeq.cu  src/                    Utils.h


In [ ]:
%%writefile FlockingBehaviourSeq.h
#include "Boid.h"
#include "Flock.h"
#include "Utils.h"
#include <stdlib.h>
#include <time.h> 
#include <iostream> 
#include <map> 
#include <vector> 

Overwriting FlockingBehaviourSeq.h


In [40]:
%%writefile FlockingBehaviourSeq.cu
#include "FlockingBehaviourSeq.h"

int main(void) {

  float velocity = 20; // boid velocity in meters per second
	double updateTime = 1.5; // update time of the simulation in seconds
	float separationWeigth = 1; // weight of the separation component in the blending
	float cohesionWeigth = 1; // weight of the cohesion component in the blending
	float alignWeigth = 1; // weight of the align component in the blending
	int flockDim = 10000000; // 10000000 number of boids in the flock
	float neighDim = 75000000; // 75000000 dimension of the neighborhood in meters
	int minRand = -50000000; // -50000000 minimum value that can be generated for initial position and direction
	int maxRand = 50000000; // 50000000 maximum value that can be generated for initial position and direction
	float decimals = 3; // 0 number of decimal digits in the generated values for initial position and direction
	int iterations = 1; // number of updates 

  Flock f{};

	srand (time(NULL));
	int div = pow(10, decimals);

	// generate boids with random initial position and direction and add them to the flock
	printf("\n\nCPU Flock generation...\n");
	double cpuTimeStart = seconds();

	std::vector<float> dir;
	float p1, p2, p3, d1, d2, d3;
	for(int i = 0; i < flockDim; i++){
			p1 = minRand + rand() % (maxRand + 1 - minRand);
			p2 = minRand + rand() % (maxRand + 1 - minRand);
			p3 = minRand + rand() % (maxRand + 1 - minRand);
			d1 = minRand + rand() % (maxRand + 1 - minRand);
			d2 = minRand + rand() % (maxRand + 1 - minRand);
			d3 = minRand + rand() % (maxRand + 1 - minRand);
			dir = std::vector<float>{d1/div, d2/div, d3/div};
			vector3Normalize(dir);
	}

	double cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

	for(int i = 0; i < flockDim; i++){
			f.addBoid(Boid{i, velocity, std::vector<float>{p1/div,p2/div,p3/div}, dir});
	}

/*
	std::vector<float> dir;
	float p1, p2, p3, d1, d2, d3;
	for(int i = 0; i < flockDim; i++){
			p1 = minRand + rand() % (maxRand + 1 - minRand);
			p2 = minRand + rand() % (maxRand + 1 - minRand);
			p3 = minRand + rand() % (maxRand + 1 - minRand);
			d1 = minRand + rand() % (maxRand + 1 - minRand);
			d2 = minRand + rand() % (maxRand + 1 - minRand);
			d3 = minRand + rand() % (maxRand + 1 - minRand);
			dir = std::vector<float>{d1/div, d2/div, d3/div};
			vector3Normalize(dir);
			f.addBoid(Boid{i, velocity, std::vector<float>{p1/div,p2/div,p3/div}, dir});
	}
*/
	cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

  // set flock parameters
  f.setBlendingWeigths(separationWeigth, cohesionWeigth, alignWeigth);
	f.setNeighborhoodDim(neighDim);

	// generate neighborhoods
	printf("\n\nCPU neighborhoods generation...\n");
	cpuTimeStart = seconds();
	f.updateNeighborhoodMap();
	cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

	//f.print();
	//std::cout << std::endl;
	//f.printNeighborhoods();

	// start simulation loop that updates the flock each updateTime
	double loopStart = seconds();
	double tmpTime = updateTime;
	while(iterations > 0){

	 		auto duration = seconds() - loopStart;
	 		if(duration >= tmpTime)
			{
				 	printf("\n\nCPU Flock update...\n");
					cpuTimeStart = seconds();
					f.updateFlock(updateTime);
					cpuTime = seconds() - cpuTimeStart;
					printf("    CPU elapsed time: %.5f (sec)", cpuTime);
				
					tmpTime += updateTime;
					iterations--;

					//std::cout << std::endl;
				  //f.print();
				
					//std::cout << std::endl;
					//f.printNeighborhoods();
			}
	}
	
	return 0;
}

Overwriting FlockingBehaviourSeq.cu


In [ ]:
%%writefile Boid.h
#include <vector>
#include <iostream>
#include <iomanip>
#include <string>

#ifndef BOID_H
#define BOID_H
class Boid{
    
    public:
    Boid();

    Boid(const int, const float, const std::vector<float>&, const std::vector<float>&);

    const std::vector<float>& getDirection() const;
    const std::vector<float>& getPosition() const;
    const int& getId() const;
    void setDirection(const std::vector<float>&);
    void setPosition(const std::vector<float>&);

    // move the boid by updating its position based on its current direction, velocity and time passed since the last update
    void move(float);

    void print() const;

    private:
    int id; // unique id of the boid
    float velocity; // boid velocity in meters per second
    std::vector<float> position; // one unit is one meter
    std::vector<float> direction; // normalized vector
};
#endif

Overwriting Boid.h


In [ ]:
%%writefile Boid.cpp
#include "Boid.h"

Boid::Boid(): id{0}, velocity{1}, direction{0,0,0}, position{0,0,0} {}

Boid::Boid(const int id, const float vel, const std::vector<float>& pos, const std::vector<float>& dir): id{id}, velocity{vel}, direction{dir}, position{pos} {}

const std::vector<float>& Boid::getDirection() const { return direction; }

const std::vector<float>& Boid::getPosition() const { return position; }

const int& Boid::getId() const { return id; }

void Boid::setDirection(const std::vector<float>& dir) { direction = dir; }

void Boid::setPosition(const std::vector<float>& pos) { position = pos; }

void Boid::move(float time) { 
    
    for(int i = 0; i < 3; i++){
        position[i] += direction[i] * velocity * time;
    }
}

void Boid::print() const{
    std::cout << std::setprecision(4) << "Boid " << id << ": " << "pos(" << position[0] << ", " << position[1] << ", " << position[2] << 
    "); dir(" << direction[0] << ", " << direction[1] << ", " << direction[2] << ")" << std::endl;
}

Writing Boid.cpp


In [ ]:
%%writefile Flock.h
#include <iostream>
#include <map>
#include <vector>
#include "Boid.h"
#include "Utils.h"

#ifndef FLOCK_H
#define FLOCK_H
class Flock{
    
    public:
    Flock();
    Flock(std::map<int, Boid>&&);

    const std::map<int, std::vector<int>>& getNeighborhoodMap() const;
    const std::map<int, Boid>& getBoidsMap() const;

    const std::vector<int>& getNeighborhood(const Boid&) const;
    void setNeighborhoodDim(const int);
    void setBlendingWeigths(const int, const int, const int);

    // returns the list of boids that are nearer than neighborhoodDim from the passed boid
    std::vector<int> computeNeighborhood(const Boid&);

    // adds a boid to the flock
    void addBoid(const Boid&);

    // computes the neighborhoods of each boid based on the current situation and puts them inside the neighborhoodMap
    void updateNeighborhoodMap();

    // for each boid computes all the components that influence its direction, blends them and set the resulting 
    // direction as the current direction of the boid only if it is not (0,0,0). otherwise the old direction is maintained
    void updateFlock(float);

    std::vector<float> getSeparationDirection(const int) const;
    std::vector<float> getCohesionDirection(const int) const;
    std::vector<float> getAlignDirection(const int) const;

    void print() const;
    void printNeighborhoods() const;

    private:
    std::map<int, std::vector<int>> neighborhoodMap; // associates the neighborhood of a boid to the id of the boid
    std::map<int, Boid> boidsMap; // associates a boid to its id
    float neighborhoodDim; // dimension of the neighborhood in meters
    float separationWeigth; // weight of the separation component in the blending
    float cohesionWeigth; // weight of the cohesion component in the blending
    float alignWeigth; // weight of the align component in the blending
};
#endif

Overwriting Flock.h


In [ ]:
%%writefile Flock.cpp
#include "Flock.h"

Flock::Flock(): boidsMap{}, neighborhoodMap{} {}

Flock::Flock(std::map<int, Boid>&& boidsMap): boidsMap{boidsMap} {
    
    updateNeighborhoodMap();
}

const std::map<int, std::vector<int>>& Flock::getNeighborhoodMap() const { return neighborhoodMap; }

const std::map<int, Boid>& Flock::getBoidsMap() const { return boidsMap; }

const std::vector<int>& Flock::getNeighborhood(const Boid& b) const { return neighborhoodMap.at(b.getId()); }

void Flock::setNeighborhoodDim(int value){ neighborhoodDim = value; }

void Flock::setBlendingWeigths(const int s, const int c, const int a){
    separationWeigth = s;
    cohesionWeigth = c;
    alignWeigth = a;
}

std::vector<int> Flock::computeNeighborhood(const Boid& b){
    
    std::vector<int> neighborhood{};
    for(const auto& elem : boidsMap){
        
        if(elem.first != b.getId() && vector3Distance(elem.second.getPosition(), b.getPosition()) <= neighborhoodDim){
            neighborhood.push_back(elem.first);
        }
    }

    return neighborhood;
}

void Flock::addBoid(const Boid& b){
    boidsMap.insert(std::pair<int,Boid>(b.getId(), b));
    neighborhoodMap.insert(std::pair<int,std::vector<int>>(b.getId(), std::vector<int>{}));
}

void Flock::updateNeighborhoodMap(){
    
    for(const auto& elem : boidsMap){
        if(neighborhoodMap.find(elem.first) != neighborhoodMap.end()){
            neighborhoodMap[elem.first] = computeNeighborhood(elem.second);
        }
        else{
            neighborhoodMap.insert(std::pair<int,std::vector<int>>(elem.first, computeNeighborhood(elem.second)));
        }
    }
}

void Flock::updateFlock(float time){
    
    std::vector<float> cohesion;
    std::vector<float> separation;
    std::vector<float> align;
    std::vector<float> finalDirection;
		for(auto& elem : boidsMap){
      
        separation = getSeparationDirection(elem.first);
        cohesion = getCohesionDirection(elem.first);
        align = getAlignDirection(elem.first);

        finalDirection = blendDirections(separation, cohesion, align);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            elem.second.setDirection(finalDirection);
        }

        elem.second.move(time);
    }

    updateNeighborhoodMap();
}

std::vector<float> Flock::getSeparationDirection(int b) const{
    
    std::vector<float> separation{0,0,0};
    std::vector<float> tmp;
    for(const auto& n : neighborhoodMap.at(b)){
        tmp = vector3Sub(boidsMap.at(b).getPosition(), boidsMap.at(n).getPosition());
        vector3Normalize(tmp);
        tmp = vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001));
        separation = vector3Sum(separation, tmp);
    }

    vector3Normalize(separation);
    return vector3Mul(separation, separationWeigth);
}

std::vector<float> Flock::getCohesionDirection(int b) const{
    
    std::vector<float> cohesion{0,0,0};
    float count = 0.0;
    for(const auto& n : neighborhoodMap.at(b)){
        cohesion = vector3Sum(cohesion, boidsMap.at(n).getPosition());
        count++;
    }

    if(count != 0){
        cohesion = vector3Mul(cohesion, 1.0/count);
        cohesion = vector3Sub(cohesion, boidsMap.at(b).getPosition());
    }

    vector3Normalize(cohesion);
    return vector3Mul(cohesion, cohesionWeigth);
}

std::vector<float> Flock::getAlignDirection(int b) const{
    
    std::vector<float> align{0,0,0};
    for(const auto& n : neighborhoodMap.at(b)){
        align = vector3Sum(align, boidsMap.at(n).getDirection());
    }

    vector3Normalize(align);
    return vector3Mul(align, alignWeigth);
}

void Flock::print() const{
     
    for(const auto& elem : boidsMap){
        elem.second.print();
    }
}

void Flock::printNeighborhoods() const{
     
    for(const auto& elem : boidsMap){
        std::cout << elem.first << ": ";
        for(const auto& elem : getNeighborhood(elem.second)){
            std::cout << elem << ", ";
        }
        std::cout << std::endl;
    }
}

Overwriting Flock.cpp


In [ ]:
%%writefile Utils.h
#include <vector>
#include <math.h>
#include <chrono>
#include <sys/time.h>

#ifndef UTILS_H
#define UTILS_H

typedef unsigned long ulong;
typedef unsigned int uint;

// returns the distance between the passed vectors
float vector3Distance(const std::vector<float>&, const std::vector<float>&);

// returns the sum of the passed vectors
std::vector<float> vector3Sum(const std::vector<float>&, const std::vector<float>&);

// returns the subtraction of the passed vectors
std::vector<float> vector3Sub(const std::vector<float>&, const std::vector<float>&);

// returns the multiplication of the passed vectors
std::vector<float> vector3Mul(const std::vector<float>&, const float);

// returns the blending of the passed vectors representing directions
std::vector<float> blendDirections(const std::vector<float>&, const std::vector<float>&, const std::vector<float>&);

// returns the magnitude of the passed vector
float vector3Magnitude(const std::vector<float>&);

// normalizes the passed vector
void vector3Normalize(std::vector<float>&);

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

#endif

Overwriting Utils.h


In [ ]:
%%writefile Utils.cpp
#include "Utils.h"

float vector3Distance(const std::vector<float>& v, const std::vector<float>& w){
    return sqrt(pow(w[0] - v[0], 2) + pow(w[1] - v[1], 2) + pow(w[2] - v[2], 2));
}

std::vector<float> vector3Sum(const std::vector<float>& v, const std::vector<float>& w){
    return std::vector<float>{v[0] + w[0], v[1] + w[1], v[2] + w[2]};
}

std::vector<float> vector3Sub(const std::vector<float>& v, const std::vector<float>& w){
    return std::vector<float>{v[0] - w[0], v[1] - w[1], v[2] - w[2]};
}

std::vector<float> vector3Mul(const std::vector<float>& v, const float n){
    return std::vector<float>{v[0] * n, v[1] * n, v[2] * n};
}

std::vector<float> blendDirections(const std::vector<float>& v, const std::vector<float>& w, const std::vector<float>& u){
    
    std::vector<float> res{0,0,0};
    res = vector3Sum(res, v);
    res = vector3Sum(res, w);
    res = vector3Sum(res, u);

    return res;
}

float vector3Magnitude(const std::vector<float>& v){
    return sqrt(pow(v[0], 2) + pow(v[1], 2) + pow(v[2], 2));
}

void vector3Normalize(std::vector<float>& v){
    if(v[0] != 0 || v[1] != 0 || v[2] != 0){
        float magnitude = vector3Magnitude(v);
        v[0] *= (1/magnitude);
        v[1] *= (1/magnitude);
        v[2] *= (1/magnitude);
    }
}

Overwriting Utils.cpp


In [39]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_37 FlockingBehaviourSeq.cu Boid.cpp Flock.cpp Utils.cpp -o FlockingBehaviourSeq

# Execution
!./FlockingBehaviourSeq

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


CPU Flock generation...
    CPU elapsed time: 21.77822 (sec)
^C


# Flocking Behaviour Parallel

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
%cd "content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel"
%ls

/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel
Boid.cpp   Flock.h                  FlockingBehaviourPar.h   Utils.cpp
Boid.h     FlockingBehaviourPar     FlockingBehaviourSeq.cu  Utils.h
Flock.cpp  FlockingBehaviourPar.cu  src/


In [7]:
%cd ..
%cd "FlockingBehaviourSequential"
%ls

/
[Errno 2] No such file or directory: 'FlockingBehaviourSequential'
/
bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [12]:
%cd ..
%cd "FlockingBehaviourParallel"
%ls

/content/drive/MyDrive/GPU Computing/Progetto
/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel
Boid.cpp  Flock.cpp  FlockingBehaviourPar     FlockingBehaviourPar.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourPar.cu  src/                    Utils.h


In [ ]:
%%writefile FlockingBehaviourPar.h
#include "Boid.h"
#include "Flock.h"
#include "Utils.h"
#include <stdlib.h>
#include <time.h> 
#include <iostream> 
#include <map> 
#include <vector> 
#include <curand_kernel.h>

Overwriting FlockingBehaviourPar.h


In [81]:
%%writefile FlockingBehaviourPar.cu
#include "FlockingBehaviourPar.h"

#define BLOCK_SIZE 384
#define GRID_SIZE 64
//512
//64

/*
 * Initialize cuRAND states
 */
__global__ void initializeStates(uint seed, curandState *states) {

	uint tid = threadIdx.x + blockDim.x * blockIdx.x;

	curand_init(seed, tid, 0, &states[tid]);
}

/*
 * Kernel for flock generation: generates boids with random initial position and direction and adds them to the flock
 */
__global__ void generateBoidsStatus(int minRand, uint minMaxDiff, int div, uint flockDim, uint threadsNum, uint generationsPerThread, float *generated, curandState *states) {

	uint tid = threadIdx.x + blockDim.x * blockIdx.x;

	if(tid < threadsNum){
			
			curandState localState = states[tid];

			//to avoid accesses out of bounds
			//if the thread is the last make it generate the numbers for the remaining boids, otherwise make it generate generationsPerThread numbers
			generationsPerThread = (tid == (threadsNum - 1)) * (flockDim - generationsPerThread * tid) + !(tid == (threadsNum - 1)) * generationsPerThread;
	
			float d1, d2, d3;
			float value;
			uint pos;
			//generationsPerThread = generationsPerThread/2;
			for(uint i = 0; i < generationsPerThread; i++){
					d1 = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					d2 = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					d3 = (curand_uniform(&localState) * minMaxDiff + minRand) / div;

					value = sqrt(d1*d1 + d2*d2 + d3*d3);

					//if the magnitude is 0 avoid dividing for 0 and set the value to 0, otherwise calculate the value to normalize
					value = !(value == 0) * 1/(value + (value == 0));

					pos = tid + i * threadsNum;
					generated[pos] = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					generated[pos + flockDim] = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					generated[pos + 2 * flockDim] = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					generated[pos + 3 * flockDim] = d1 * value;
					generated[pos + 4 * flockDim] = d2 * value;
					generated[pos + 5 * flockDim] = d3 * value;
					
					/*
					d1 = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					d2 = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					d3 = (curand_uniform(&localState) * minMaxDiff + minRand) / div;

					magnitude = sqrt(d1*d1 + d2*d2 + d3*d3);

					//if the magnitude is 0 avoid dividing for 0 and set the value to 0, otherwise calculate the value to normalize
					value = !(magnitude == 0) * 1/(magnitude + (magnitude == 0));

					pos = tid + (i+generationsPerThread) * threadsNum;
					generated[pos] = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					generated[pos + flockDim] = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					generated[pos + 2 * flockDim] = (curand_uniform(&localState) * minMaxDiff + minRand) / div;
					generated[pos + 3 * flockDim] = d1 * value;
					generated[pos + 4 * flockDim] = d2 * value;
					generated[pos + 5 * flockDim] = d3 * value;
					*/
			}		
			
			states[tid] = localState;
	}
}

int main(void) {

  float velocity = 20; // boid velocity in meters per second
	double updateTime = 1.5; // update time of the simulation in seconds
	float separationWeigth = 1; // weight of the separation component in the blending
	float cohesionWeigth = 1; // weight of the cohesion component in the blending
	float alignWeigth = 1; // weight of the align component in the blending
	int flockDim = 100000000; // 10000000 number of boids in the flock
	float neighDim = 75000000; // 75000000 dimension of the neighborhood in meters
	int minRand = -50000000; // -50000000 minimum value that can be generated for initial position and direction
	int maxRand = 50000000; // 50000000 maximum value that can be generated for initial position and direction
	float decimals = 3; // 0 number of decimal digits in the generated values for initial position and direction
	int iterations = 1; // number of updates 

	device_name();

	// events to measure time
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	float milliseconds = 0;

	// prepare for flock generation
  Flock f{};
	int div = pow(10, decimals);
	int numsToGenerate = flockDim * 6;

	float *generatedNumsHost = (float *) malloc(numsToGenerate * sizeof(float));
	float *generatedNumsDev;
	curandState *states;
	CHECK(cudaMalloc((void **) &generatedNumsDev, numsToGenerate * sizeof(float)));
	int blockSize = BLOCK_SIZE;
	int gridSize = GRID_SIZE; 
	int generationsPerThread = 5000; 
	int threadsNum = (flockDim + generationsPerThread - 1) / generationsPerThread;

	if(threadsNum > blockSize * gridSize){
			std::cout << "Not enough threads" << std::endl;
	}

	CHECK(cudaMalloc((void **) &states, blockSize * gridSize * sizeof(curandState)));
	cudaEventRecord(start);

	// generate flock
	printf("\n\nGPU Flock generation...\n");

	initializeStates<<<gridSize, blockSize>>>(time(NULL), states);

  generateBoidsStatus<<<gridSize, blockSize>>>(minRand, maxRand - minRand, div, flockDim, threadsNum, generationsPerThread, generatedNumsDev, states);

	cudaEventRecord(stop);
	CHECK(cudaEventSynchronize(stop));
	cudaEventElapsedTime(&milliseconds, start, stop);
	printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

	CHECK(cudaMemcpy(generatedNumsHost, generatedNumsDev, numsToGenerate * sizeof(float), cudaMemcpyDeviceToHost));

	printf("threadsNum: %i\n", threadsNum);
	printf("effthreadsNum: %i\n", blockSize * gridSize);
  printf("genperthread: %i\n", generationsPerThread);

	for(int i = 0; i < 12; i++){
			printf("first: %.5f\n", generatedNumsHost[i]);
	}
	for(int i = 0; i < 12; i++){
			printf("last: %.5f\n", generatedNumsHost[numsToGenerate - i - 1]);
	}

	std::vector<float> dir;
	float p1, p2, p3, d1, d2, d3;
	uint threadGenerations;
	bool a = true;
	for(int i = 0; i < threadsNum; i++){

		  if(i == (threadsNum - 1)){
				
				  threadGenerations = flockDim - generationsPerThread * i;
		  }
		  else{
				
				  threadGenerations = generationsPerThread;
		  }
			
			for(int j = 0; j < threadGenerations; j++){
					
					p1 = generatedNumsHost[i + j * threadsNum];
					p2 = generatedNumsHost[i + j * threadsNum + flockDim];
					p3 = generatedNumsHost[i + j * threadsNum + 2 * flockDim];
					d1 = generatedNumsHost[i + j * threadsNum + 3 * flockDim];
					d2 = generatedNumsHost[i + j * threadsNum + 4 * flockDim];
					d3 = generatedNumsHost[i + j * threadsNum + 5 * flockDim];
					f.addBoid(Boid{i/6, velocity, std::vector<float>{p1,p2,p3}, std::vector<float>{d1,d2,d3}});

					if(a){
							
							printf("p1: %.5f\n", p1);
							printf("p2: %.5f\n", p2);
							printf("p3: %.5f\n", p3);
							printf("d1: %.5f\n", d1);
							printf("d2: %.5f\n", d2);
							printf("d3: %.5f\n", d3);
							a = false;
					}
			}
	}

	cudaEventRecord(stop);
	CHECK(cudaEventSynchronize(stop));
	cudaEventElapsedTime(&milliseconds, start, stop);
	printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

	CHECK(cudaFree(generatedNumsDev));
	CHECK(cudaFree(states));

  // set flock parameters
	f.setBlendingWeigths(separationWeigth, cohesionWeigth, alignWeigth);
	f.setNeighborhoodDim(neighDim);

  // generate neighborhoods
	printf("\n\nCPU neighborhoods generation...\n");
	double gpuTimeStart = seconds();
	f.updateNeighborhoodMap();
	double gpuTime = seconds() - gpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", gpuTime);

	//f.print();
	//std::cout << std::endl;
	//f.printNeighborhoods();

	// start simulation loop that updates the flock each updateTime
	double loopStart = seconds();
	double tmpTime = updateTime;
	while(iterations > 0){

	 		auto duration = seconds() - loopStart;
	 		if(duration >= tmpTime)
			{
				 	printf("\n\nGPU Flock update...\n");
					gpuTimeStart = seconds();
					f.updateFlock(updateTime);
					gpuTime = seconds() - gpuTimeStart;
					printf("    GPU elapsed time: %.5f (sec)", gpuTime);
				
					tmpTime += updateTime;
					iterations--;

					//std::cout << std::endl;
				  //f.print();
				
					//std::cout << std::endl;
					//f.printNeighborhoods();
			}
	}
	
	return 0;
}

Overwriting FlockingBehaviourPar.cu


In [ ]:
%%writefile Boid.h
#include <vector>
#include <iostream>
#include <iomanip>
#include <string>

#ifndef BOID_H
#define BOID_H
class Boid{
    
    public:
    Boid();

    Boid(const int, const float, const std::vector<float>&, const std::vector<float>&);

    const std::vector<float>& getDirection() const;
    const std::vector<float>& getPosition() const;
    const int& getId() const;
    void setDirection(const std::vector<float>&);
    void setPosition(const std::vector<float>&);

    // move the boid by updating its position based on its current direction, velocity and time passed since the last update
    void move(float);

    void print() const;

    private:
    int id; // unique id of the boid
    float velocity; // boid velocity in meters per second
    std::vector<float> position; // one unit is one meter
    std::vector<float> direction; // normalized vector
};
#endif

Writing Boid.h


In [ ]:
%%writefile Boid.cpp
#include "Boid.h"

Boid::Boid(): id{0}, velocity{1}, direction{0,0,0}, position{0,0,0} {}

Boid::Boid(const int id, const float vel, const std::vector<float>& pos, const std::vector<float>& dir): id{id}, velocity{vel}, direction{dir}, position{pos} {}

const std::vector<float>& Boid::getDirection() const { return direction; }

const std::vector<float>& Boid::getPosition() const { return position; }

const int& Boid::getId() const { return id; }

void Boid::setDirection(const std::vector<float>& dir) { direction = dir; }

void Boid::setPosition(const std::vector<float>& pos) { position = pos; }

void Boid::move(float time) { 
    
    for(int i = 0; i < 3; i++){
        position[i] += direction[i] * velocity * time;
    }
}

void Boid::print() const{
    std::cout << std::setprecision(4) << "Boid " << id << ": " << "pos(" << position[0] << ", " << position[1] << ", " << position[2] << 
    "); dir(" << direction[0] << ", " << direction[1] << ", " << direction[2] << ")" << std::endl;
}

Writing Boid.cpp


In [ ]:
%%writefile Flock.h
#include <iostream>
#include <map>
#include <vector>
#include "Boid.h"
#include "Utils.h"

#ifndef FLOCK_H
#define FLOCK_H
class Flock{
    
    public:
    Flock();
    Flock(std::map<int, Boid>&&);

    const std::map<int, std::vector<int>>& getNeighborhoodMap() const;
    const std::map<int, Boid>& getBoidsMap() const;

    const std::vector<int>& getNeighborhood(const Boid&) const;
    void setNeighborhoodDim(const int);
    void setBlendingWeigths(const int, const int, const int);

    // returns the list of boids that are nearer than neighborhoodDim from the passed boid
    std::vector<int> computeNeighborhood(const Boid&);

    // adds a boid to the flock
    void addBoid(const Boid&);

    // computes the neighborhoods of each boid based on the current situation and puts them inside the neighborhoodMap
    void updateNeighborhoodMap();

    // for each boid computes all the components that influence its direction, blends them and set the resulting 
    // direction as the current direction of the boid only if it is not (0,0,0). otherwise the old direction is maintained
    void updateFlock(float);

    std::vector<float> getSeparationDirection(const int) const;
    std::vector<float> getCohesionDirection(const int) const;
    std::vector<float> getAlignDirection(const int) const;

    void print() const;
    void printNeighborhoods() const;

    private:
    std::map<int, std::vector<int>> neighborhoodMap; // associates the neighborhood of a boid to the id of the boid
    std::map<int, Boid> boidsMap; // associates a boid to its id
    float neighborhoodDim; // dimension of the neighborhood in meters
    float separationWeigth; // weight of the separation component in the blending
    float cohesionWeigth; // weight of the cohesion component in the blending
    float alignWeigth; // weight of the align component in the blending
};
#endif

Writing Flock.h


In [ ]:
%%writefile Flock.cpp
#include "Flock.h"

Flock::Flock(): boidsMap{}, neighborhoodMap{} {}

Flock::Flock(std::map<int, Boid>&& boidsMap): boidsMap{boidsMap} {
    
    updateNeighborhoodMap();
}

const std::map<int, std::vector<int>>& Flock::getNeighborhoodMap() const { return neighborhoodMap; }

const std::map<int, Boid>& Flock::getBoidsMap() const { return boidsMap; }

const std::vector<int>& Flock::getNeighborhood(const Boid& b) const { return neighborhoodMap.at(b.getId()); }

void Flock::setNeighborhoodDim(int value){ neighborhoodDim = value; }

void Flock::setBlendingWeigths(const int s, const int c, const int a){
    separationWeigth = s;
    cohesionWeigth = c;
    alignWeigth = a;
}

std::vector<int> Flock::computeNeighborhood(const Boid& b){
    
    std::vector<int> neighborhood{};
    for(const auto& elem : boidsMap){
        
        if(elem.first != b.getId() && vector3Distance(elem.second.getPosition(), b.getPosition()) <= neighborhoodDim){
            neighborhood.push_back(elem.first);
        }
    }

    return neighborhood;
}

void Flock::addBoid(const Boid& b){
    boidsMap.insert(std::pair<int,Boid>(b.getId(), b));
    neighborhoodMap.insert(std::pair<int,std::vector<int>>(b.getId(), std::vector<int>{}));
}

void Flock::updateNeighborhoodMap(){
    
    for(const auto& elem : boidsMap){
        if(neighborhoodMap.find(elem.first) != neighborhoodMap.end()){
            neighborhoodMap[elem.first] = computeNeighborhood(elem.second);
        }
        else{
            neighborhoodMap.insert(std::pair<int,std::vector<int>>(elem.first, computeNeighborhood(elem.second)));
        }
    }
}

void Flock::updateFlock(float time){
    
    std::vector<float> cohesion;
    std::vector<float> separation;
    std::vector<float> align;
    std::vector<float> finalDirection;
		for(auto& elem : boidsMap){
      
        separation = getSeparationDirection(elem.first);
        cohesion = getCohesionDirection(elem.first);
        align = getAlignDirection(elem.first);

        finalDirection = blendDirections(separation, cohesion, align);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            elem.second.setDirection(finalDirection);
        }

        elem.second.move(time);
    }

    updateNeighborhoodMap();
}

std::vector<float> Flock::getSeparationDirection(int b) const{
    
    std::vector<float> separation{0,0,0};
    std::vector<float> tmp;
    for(const auto& n : neighborhoodMap.at(b)){
        tmp = vector3Sub(boidsMap.at(b).getPosition(), boidsMap.at(n).getPosition());
        vector3Normalize(tmp);
        tmp = vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001));
        separation = vector3Sum(separation, tmp);
    }

    vector3Normalize(separation);
    return vector3Mul(separation, separationWeigth);
}

std::vector<float> Flock::getCohesionDirection(int b) const{
    
    std::vector<float> cohesion{0,0,0};
    float count = 0.0;
    for(const auto& n : neighborhoodMap.at(b)){
        cohesion = vector3Sum(cohesion, boidsMap.at(n).getPosition());
        count++;
    }

    if(count != 0){
        cohesion = vector3Mul(cohesion, 1.0/count);
        cohesion = vector3Sub(cohesion, boidsMap.at(b).getPosition());
    }

    vector3Normalize(cohesion);
    return vector3Mul(cohesion, cohesionWeigth);
}

std::vector<float> Flock::getAlignDirection(int b) const{
    
    std::vector<float> align{0,0,0};
    for(const auto& n : neighborhoodMap.at(b)){
        align = vector3Sum(align, boidsMap.at(n).getDirection());
    }

    vector3Normalize(align);
    return vector3Mul(align, alignWeigth);
}

void Flock::print() const{
     
    for(const auto& elem : boidsMap){
        elem.second.print();
    }
}

void Flock::printNeighborhoods() const{
     
    for(const auto& elem : boidsMap){
        std::cout << elem.first << ": ";
        for(const auto& elem : getNeighborhood(elem.second)){
            std::cout << elem << ", ";
        }
        std::cout << std::endl;
    }
}

Overwriting Flock.cpp


In [ ]:
%%writefile Utils.h
#include <vector>
#include <math.h>
#include <chrono>
#include <sys/time.h>
#include <cuda_runtime_api.h>
#include <cuda.h>
#include <stdio.h>

#ifndef UTILS_H
#define UTILS_H

typedef unsigned long ulong;
typedef unsigned int uint;

// returns the distance between the passed vectors
float vector3Distance(const std::vector<float>&, const std::vector<float>&);

// returns the sum of the passed vectors
std::vector<float> vector3Sum(const std::vector<float>&, const std::vector<float>&);

// returns the subtraction of the passed vectors
std::vector<float> vector3Sub(const std::vector<float>&, const std::vector<float>&);

// returns the multiplication of the passed vectors
std::vector<float> vector3Mul(const std::vector<float>&, const float);

// returns the blending of the passed vectors representing directions
std::vector<float> blendDirections(const std::vector<float>&, const std::vector<float>&, const std::vector<float>&);

// returns the magnitude of the passed vector
float vector3Magnitude(const std::vector<float>&);

// normalizes the passed vector
void vector3Normalize(std::vector<float>&);

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
    }                                                                          \
}

#define CHECK_CUBLAS(call)                                                     \
{                                                                              \
    cublasStatus_t err;                                                        \
    if ((err = (call)) != CUBLAS_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CUBLAS error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CURAND(call)                                                     \
{                                                                              \
    curandStatus_t err;                                                        \
    if ((err = (call)) != CURAND_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CURAND error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUFFT(call)                                                      \
{                                                                              \
    cufftResult err;                                                           \
    if ( (err = (call)) != CUFFT_SUCCESS)                                      \
    {                                                                          \
        fprintf(stderr, "Got CUFFT error %d at %s:%d\n", err, __FILE__,        \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUSPARSE(call)                                                   \
{                                                                              \
    cusparseStatus_t err;                                                      \
    if ((err = (call)) != CUSPARSE_STATUS_SUCCESS)                             \
    {                                                                          \
        fprintf(stderr, "Got error %d at %s:%d\n", err, __FILE__, __LINE__);   \
        cudaError_t cuda_err = cudaGetLastError();                             \
        if (cuda_err != cudaSuccess)                                           \
        {                                                                      \
            fprintf(stderr, "  CUDA error \"%s\" also detected\n",             \
                    cudaGetErrorString(cuda_err));                             \
        }                                                                      \
        exit(1);                                                               \
    }                                                                          \
}

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

inline void device_name() {
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));
}

#endif

Overwriting Utils.h


In [ ]:
%%writefile Utils.cpp
#include "Utils.h"

float vector3Distance(const std::vector<float>& v, const std::vector<float>& w){
    return sqrt(pow(w[0] - v[0], 2) + pow(w[1] - v[1], 2) + pow(w[2] - v[2], 2));
}

std::vector<float> vector3Sum(const std::vector<float>& v, const std::vector<float>& w){
    return std::vector<float>{v[0] + w[0], v[1] + w[1], v[2] + w[2]};
}

std::vector<float> vector3Sub(const std::vector<float>& v, const std::vector<float>& w){
    return std::vector<float>{v[0] - w[0], v[1] - w[1], v[2] - w[2]};
}

std::vector<float> vector3Mul(const std::vector<float>& v, const float n){
    return std::vector<float>{v[0] * n, v[1] * n, v[2] * n};
}

std::vector<float> blendDirections(const std::vector<float>& v, const std::vector<float>& w, const std::vector<float>& u){
    
    std::vector<float> res{0,0,0};
    res = vector3Sum(res, v);
    res = vector3Sum(res, w);
    res = vector3Sum(res, u);

    return res;
}

float vector3Magnitude(const std::vector<float>& v){
    return sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2]);
}

void vector3Normalize(std::vector<float>& v){
    if(v[0] != 0 || v[1] != 0 || v[2] != 0){
        float magnitude = vector3Magnitude(v);
        v[0] *= (1/magnitude);
        v[1] *= (1/magnitude);
        v[2] *= (1/magnitude);
    }
}

Overwriting Utils.cpp


In [13]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_75 FlockingBehaviourPar.cu Boid.cpp Flock.cpp Utils.cpp -lcurand -o FlockingBehaviourPar

# Execution
!./FlockingBehaviourPar

device 0: Tesla T4
tcmalloc: large alloc 2400002048 bytes == 0x556a3e3e2000 @  0x7f53768c71e7 0x556a3c880c5f 0x7f537555abf7 0x556a3c880a0a


GPU Flock generation...
    GPU elapsed time: 0.02311 (sec)
threadsNum: 20000
effthreadsNum: 24576
genperthread: 5000
first: 31986.70117
first: -20999.52148
first: 41111.54688
first: -20097.97070
first: -29356.00195
first: -46842.33594
first: -7662.53174
first: 24983.43555
first: 36435.56641
first: -14053.27930
first: -470.65912
first: 48518.09766
last: -0.65163
last: -0.09194
last: 0.29723
last: -0.08443
last: 0.99469
last: 0.60045
last: -0.63661
last: -0.26967
last: -0.49948
last: 0.20567
last: -0.25458
last: 0.71505
p1: 31986.70117
p2: -41961.21875
p3: 30048.49609
d1: 0.06888
d2: -0.41502
d3: -0.90720
^C
